# Libraries&Function 

In [1]:
from sklearn.preprocessing import FunctionTransformer

from reskit.norms import binar_norm, wbysqdist
from reskit.norms import spectral_norm

from reskit.features import degrees,  pagerank

from reskit.core import Transformer, Pipeliner

from sklearn.feature_selection import VarianceThreshold

from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier 
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import os
import pandas as pd
import numpy as np

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

def orig(x):
    return x

import copy

In [2]:
def get_autism(path_to_read='Data/dti/', distances=True):
    def get_autism_distances(loc_name):
        with open(loc_name, 'r') as f:
            read_data = f.readlines()

        read_data = pd.DataFrame(
            np.array([np.array(item[:-1].split()).astype(int) for item in read_data]))

        return read_data

    def get_distance_matrix(coords):
        if type(coords) == pd.core.frame.DataFrame:
            coords = coords.values
        elif type(coords) != np.ndarray:
            print('Provide either pandas df or numpy array!')
            return -1

        shape = len(coords)
        dist_matrix = np.zeros((shape, shape))
        del shape
        for i in range(len(coords)):
            for j in range(i + 1, len(coords)):
                dist_matrix[i, j] = np.linalg.norm(coords[i, :] - coords[j, :])
                dist_matrix[j, i] = dist_matrix[i, j]
        return dist_matrix

    target_vector = []  # this will be a target vector (diagnosis)
    matrices = []  # this will be a list of connectomes
    all_files = sorted(os.listdir(path_to_read))
    matrix_files = [
        item for item in all_files if 'DTI_connectivity' in item and 'All' not in item]
    distance_files = [
        item for item in all_files if 'DTI_region_xyz_centers' in item and 'All' not in item]

    # for each file in a sorted (!) list of files:
    for filename in matrix_files:

        A_dataframe = pd.read_csv(
            path_to_read + filename, sep='   ', header=None, engine='python')
        A = A_dataframe.values  # we will use a list of numpy arrays, NOT pandas dataframes
        matrices.append(A)# append a matrix to our list
        if "ASD" in filename:
            target_vector.append(1)
        elif "TD" in filename:
            target_vector.append(0)
    asd_dict = {}
    asd_dict['X'] = np.array(matrices)
    asd_dict['y'] = np.array(target_vector)
    if distances:
        dist_matrix_list = []
        for item in distance_files:
            # print(item)
            cur_coord = get_autism_distances(path_to_read + item)
            cur_dist_mtx = get_distance_matrix(cur_coord)
            dist_matrix_list += [cur_dist_mtx]

        asd_dict['dist'] = np.array(dist_matrix_list)

    return asd_dict

import matrix_eig as me

# Train mosters
## Logistic Regression
### Chose the best model for each k

In [3]:
grid_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=0)

eval_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=1)

data = [('UCLAsource', Transformer(get_autism))]

weighters = [('binar', Transformer(binar_norm))]

normalizers = [('origN', Transformer(orig))]

featurizers = [('origF', Transformer(me.orig_vec, collect=['X_vec'])),
               ('low0', Transformer(me.matrix_eig_0, collect=['X_vec'])),
               ('low20', Transformer(me.matrix_eig_20, collect=['X_vec'])),
               ('low40', Transformer(me.matrix_eig_40, collect=['X_vec'])),
               ('low60', Transformer(me.matrix_eig_60, collect=['X_vec'])),
               ('low80', Transformer(me.matrix_eig_80, collect=['X_vec'])),
               ('low100', Transformer(me.matrix_eig_100, collect=['X_vec'])),
               ('low120', Transformer(me.matrix_eig_120, collect=['X_vec'])),
               ('low140', Transformer(me.matrix_eig_140, collect=['X_vec'])),
               ('low160', Transformer(me.matrix_eig_160, collect=['X_vec'])),
               ('low180', Transformer(me.matrix_eig_180, collect=['X_vec'])),
               ('low200', Transformer(me.matrix_eig_200, collect=['X_vec'])),
               ('low220', Transformer(me.matrix_eig_220, collect=['X_vec'])),
               ('low230', Transformer(me.matrix_eig_230, collect=['X_vec'])),
               ('low240', Transformer(me.matrix_eig_240, collect=['X_vec'])),
               ('low250', Transformer(me.matrix_eig_250, collect=['X_vec'])),
               ('low260', Transformer(me.matrix_eig_260, collect=['X_vec'])),
               ('low263', Transformer(me.matrix_eig_263, collect=['X_vec']))
              ]

selectors = [('var_threshold', VarianceThreshold())]

scalers = [('minmax', MinMaxScaler())]

classifiers = [('LR', LogisticRegression())]

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

param_grid = dict(
     LR=dict(
        C=[0.01, 0.05, 0.1] + [0.05*i for i in range(3, 21)],
        max_iter=[50, 100, 500],
        penalty=['l1', 'l2']
    )
    )

banned_combos = []

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
pipe.plan_table

,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers
0,UCLAsource,binar,origN,origF,var_threshold,minmax,LR
1,UCLAsource,binar,origN,low0,var_threshold,minmax,LR
2,UCLAsource,binar,origN,low20,var_threshold,minmax,LR
3,UCLAsource,binar,origN,low40,var_threshold,minmax,LR
4,UCLAsource,binar,origN,low60,var_threshold,minmax,LR
5,UCLAsource,binar,origN,low80,var_threshold,minmax,LR
6,UCLAsource,binar,origN,low100,var_threshold,minmax,LR
7,UCLAsource,binar,origN,low120,var_threshold,minmax,LR
8,UCLAsource,binar,origN,low140,var_threshold,minmax,LR
9,UCLAsource,binar,origN,low160,var_threshold,minmax,LR


In [6]:
result = pipe.get_results('Data/dti/', caching_steps=['Data', 'Weighters', 'Normalizers', 'Featurizers'],
                          scoring=['roc_auc'],  results_file = "Results/Ver3.6/result_1.csv")

No previous results found.
Line: 1/30
Line: 2/30
Line: 3/30
Line: 4/30
Line: 5/30
Line: 6/30
Line: 7/30
Line: 8/30
Line: 9/30
Line: 10/30
Line: 11/30
Line: 12/30
Line: 13/30
Line: 14/30
Line: 15/30
Line: 16/30
Line: 17/30
Line: 18/30
Line: 19/30
Line: 20/30
Line: 21/30
Line: 22/30
Line: 23/30
Line: 24/30
Line: 25/30
Line: 26/30
Line: 27/30
Line: 28/30
Line: 29/30
Line: 30/30


In [4]:
result1 = pd.read_csv("Results/Ver3/result_LR.csv")
result1

,Unnamed: 0,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers,grid_roc_auc_mean,grid_roc_auc_std,grid_roc_auc_best_params,eval_roc_auc_mean,eval_roc_auc_std,eval_roc_auc_scores
0,0,UCLAsource,binar,origN,origF,var_threshold,minmax,LR,0.674113,0.112793,"{'penalty': 'l1', 'C': 0.2, 'max_iter': 50}",0.514000,0.154544,[ 0.8 0.72 0.32 0.55 0.35 0.35 0.55 0....
1,1,UCLAsource,binar,origN,low0,var_threshold,minmax,LR,0.640957,0.157197,"{'penalty': 'l1', 'C': 0.2, 'max_iter': 50}",0.567000,0.174014,[ 0.4 0.96 0.56 0.35 0.65 0.6 0.5 0....
2,2,UCLAsource,binar,origN,low20,var_threshold,minmax,LR,0.714539,0.138316,"{'penalty': 'l1', 'C': 0.30000000000000004, 'm...",0.641667,0.212426,[ 0.56666667 1. 0.6 0.6 ...
3,3,UCLAsource,binar,origN,low40,var_threshold,minmax,LR,0.710816,0.108996,"{'penalty': 'l1', 'C': 0.2, 'max_iter': 50}",0.567000,0.174014,[ 0.4 0.96 0.56 0.35 0.65 0.6 0.5 0....
4,4,UCLAsource,binar,origN,low60,var_threshold,minmax,LR,0.654255,0.281548,"{'penalty': 'l1', 'C': 0.30000000000000004, 'm...",0.660667,0.201471,[ 0.56666667 1. 0.64 0.6 ...
5,5,UCLAsource,binar,origN,low80,var_threshold,minmax,LR,0.745035,0.198154,"{'penalty': 'l1', 'C': 0.30000000000000004, 'm...",0.646667,0.211975,[ 0.56666667 1. 0.6 0.65 ...
6,6,UCLAsource,binar,origN,low100,var_threshold,minmax,LR,0.725177,0.158963,"{'penalty': 'l1', 'C': 0.30000000000000004, 'm...",0.642667,0.213192,[ 0.56666667 1. 0.56 0.65 ...
7,7,UCLAsource,binar,origN,low120,var_threshold,minmax,LR,0.681915,0.227831,"{'penalty': 'l1', 'C': 0.2, 'max_iter': 50}",0.655000,0.204805,[ 0.6 0.96 0.84 0.75 0.25 0.6 0.85 0....
8,8,UCLAsource,binar,origN,low140,var_threshold,minmax,LR,0.728546,0.204206,"{'penalty': 'l1', 'C': 0.30000000000000004, 'm...",0.712667,0.166565,[ 0.76666667 1. 0.56 0.75 ...
9,9,UCLAsource,binar,origN,low160,var_threshold,minmax,LR,0.647163,0.120142,"{'penalty': 'l1', 'C': 0.35000000000000003, 'm...",0.690667,0.162657,[ 0.76666667 0.88 0.76 0.95 ...


In [6]:
result2 = pd.read_csv("Results/Ver3.1/result_1.csv")
result2

,Unnamed: 0,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers,grid_roc_auc_mean,grid_roc_auc_std,grid_roc_auc_best_params,eval_roc_auc_mean,eval_roc_auc_std,eval_roc_auc_scores
0,0,UCLAsource,binar,origN,origF,var_threshold,minmax,LR,0.557801,0.111239,"{'penalty': 'l1', 'C': 0.5, 'max_iter': 50}",0.510000,0.125778,[ 0.5 0.64 0.36 0.5 0.55 0.45 0.75 0....
1,1,UCLAsource,binar,origN,low0,var_threshold,minmax,LR,0.646631,0.141233,"{'penalty': 'l1', 'C': 0.2, 'max_iter': 50}",0.567000,0.174014,[ 0.4 0.96 0.56 0.35 0.65 0.6 0.5 0....
2,2,UCLAsource,binar,origN,low5,var_threshold,minmax,LR,0.646631,0.141233,"{'penalty': 'l1', 'C': 0.2, 'max_iter': 50}",0.567000,0.174014,[ 0.4 0.96 0.56 0.35 0.65 0.6 0.5 0....
3,3,UCLAsource,binar,origN,low10,var_threshold,minmax,LR,0.646631,0.141233,"{'penalty': 'l1', 'C': 0.2, 'max_iter': 50}",0.567000,0.174014,[ 0.4 0.96 0.56 0.35 0.65 0.6 0.5 0....
4,4,UCLAsource,binar,origN,low20,var_threshold,minmax,LR,0.646631,0.141233,"{'penalty': 'l1', 'C': 0.2, 'max_iter': 50}",0.567000,0.174014,[ 0.4 0.96 0.56 0.35 0.65 0.6 0.5 0....
5,5,UCLAsource,binar,origN,low30,var_threshold,minmax,LR,0.646631,0.141233,"{'penalty': 'l1', 'C': 0.2, 'max_iter': 50}",0.567000,0.174014,[ 0.4 0.96 0.56 0.35 0.65 0.6 0.5 0....
6,6,UCLAsource,binar,origN,low40,var_threshold,minmax,LR,0.646631,0.141233,"{'penalty': 'l1', 'C': 0.2, 'max_iter': 50}",0.567000,0.174014,[ 0.4 0.96 0.56 0.35 0.65 0.6 0.5 0....
7,7,UCLAsource,binar,origN,low50,var_threshold,minmax,LR,0.646631,0.141233,"{'penalty': 'l1', 'C': 0.2, 'max_iter': 50}",0.567000,0.174014,[ 0.4 0.96 0.56 0.35 0.65 0.6 0.5 0....
8,8,UCLAsource,binar,origN,low60,var_threshold,minmax,LR,0.646631,0.141233,"{'penalty': 'l1', 'C': 0.2, 'max_iter': 50}",0.567000,0.174014,[ 0.4 0.96 0.56 0.35 0.65 0.6 0.5 0....
9,9,UCLAsource,binar,origN,low70,var_threshold,minmax,LR,0.646631,0.141233,"{'penalty': 'l1', 'C': 0.2, 'max_iter': 50}",0.567000,0.174014,[ 0.4 0.96 0.56 0.35 0.65 0.6 0.5 0....


In [5]:
grid_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=0)

eval_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=1)

data = [('UCLAsource', Transformer(get_autism))]

weighters = [('binar', Transformer(binar_norm))]

normalizers = [('origN', Transformer(orig))]

featurizers = [('origF', Transformer(me.orig_vec, collect=['X_vec'])),
               ('low0', Transformer(me.matrix_eig_0, collect=['X_vec'])),
               ('low20', Transformer(me.matrix_eig_20, collect=['X_vec'])),
               ('low40', Transformer(me.matrix_eig_40, collect=['X_vec'])),
               ('low60', Transformer(me.matrix_eig_60, collect=['X_vec'])),
               ('low80', Transformer(me.matrix_eig_80, collect=['X_vec'])),
               ('low100', Transformer(me.matrix_eig_100, collect=['X_vec'])),
               ('low120', Transformer(me.matrix_eig_120, collect=['X_vec'])),
               ('low140', Transformer(me.matrix_eig_140, collect=['X_vec'])),
               ('low160', Transformer(me.matrix_eig_160, collect=['X_vec'])),
               ('low180', Transformer(me.matrix_eig_180, collect=['X_vec'])),
               ('low200', Transformer(me.matrix_eig_200, collect=['X_vec'])),
               ('low210', Transformer(me.matrix_eig_210, collect=['X_vec'])),
               ('low220', Transformer(me.matrix_eig_220, collect=['X_vec'])),
               ('low230', Transformer(me.matrix_eig_230, collect=['X_vec'])),
               ('low240', Transformer(me.matrix_eig_240, collect=['X_vec'])),
               ('low250', Transformer(me.matrix_eig_250, collect=['X_vec'])),
               ('low260', Transformer(me.matrix_eig_260, collect=['X_vec'])),
               ('low263', Transformer(me.matrix_eig_263, collect=['X_vec']))
              ]

selectors = [('var_threshold', VarianceThreshold())]

scalers = [('minmax', MinMaxScaler())]

classifiers = [('LR_orig', LogisticRegression()),
               ('LR_0', LogisticRegression()),
               ('LR_20', LogisticRegression()),
               ('LR_40', LogisticRegression()),
               ('LR_60', LogisticRegression()),
               ('LR_80', LogisticRegression()),
               ('LR_100', LogisticRegression()),
               ('LR_120', LogisticRegression()),
               ('LR_140', LogisticRegression()),
               ('LR_160', LogisticRegression()),
               ('LR_180', LogisticRegression()),
               ('LR_200', LogisticRegression()),
               ('LR_210', LogisticRegression()),
               ('LR_220', LogisticRegression()),
               ('LR_230', LogisticRegression()),
               ('LR_240', LogisticRegression()),
               ('LR_250', LogisticRegression()),
               ('LR_260', LogisticRegression()),
               ('LR_263', LogisticRegression()),
              ]

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

param_grid = dict(
    LR_orig = dict(penalty= ['l1'], C= [0.5], max_iter= [50]),
    LR_0    = dict(penalty= ['l1'], C= [0.2], max_iter= [500]),
    LR_20   = dict(penalty= ['l1'], C= [0.2], max_iter= [50]),
    LR_40   = dict(penalty= ['l1'], C= [0.2], max_iter= [100]),
    LR_60   = dict(penalty= ['l1'], C= [0.2], max_iter= [50]),
    LR_80   = dict(penalty= ['l1'], C= [0.2], max_iter= [50]),
    LR_100  = dict(penalty= ['l1'], C= [0.2], max_iter= [100]),
    LR_120  = dict(penalty= ['l1'], C= [0.2], max_iter= [50]),
    LR_140   = dict(penalty= ['l1'], C= [0.2], max_iter= [50]),
    LR_160   = dict(penalty= ['l1'], C= [0.3], max_iter= [500]),
    LR_180   = dict(penalty= ['l1'], C= [0.3], max_iter= [100]),
    LR_200   = dict(penalty= ['l1'], C= [0.3], max_iter= [50]),
    LR_210  = dict(penalty= ['l1'], C= [0.3], max_iter= [50]),
    LR_220  = dict(penalty= ['l1'], C= [0.3], max_iter= [50]),  ##New
    LR_230  = dict(penalty= ['l1'], C= [0.3], max_iter= [50]), ##New
    LR_240  = dict(penalty= ['l1'], C= [0.4], max_iter= [100]), ##New
    LR_250  = dict(penalty= ['l2'], C= [0.4], max_iter= [50]),  ##New
    LR_260  = dict(penalty= ['l2'], C= [0.4], max_iter= [50]),
    LR_263  = dict(penalty= ['l1'], C= [0.8], max_iter= [50])   ##New
    )

banned_combos = []

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

In [6]:
k = [0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 210, 220, 230, 240, 250, 260, 263]
#col = result.as_matrix(["Featurizers"])
col_f = ['low' + str(i) for i in k]

for n,fet in enumerate(col_f):
    k_tmp = copy.copy(k)
    k_tmp.remove(k[n])
    for i in k_tmp: banned_combos.append((fet, 'LR_' + str(i)))
        
for i in k: banned_combos.append(('origF', 'LR_' + str(i)))
    
for fet in col_f: banned_combos.append((fet, 'LR_orig'))

In [7]:
pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
pipe.plan_table

,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers
0,UCLAsource,binar,origN,origF,var_threshold,minmax,LR_orig
1,UCLAsource,binar,origN,low0,var_threshold,minmax,LR_0
2,UCLAsource,binar,origN,low20,var_threshold,minmax,LR_20
3,UCLAsource,binar,origN,low40,var_threshold,minmax,LR_40
4,UCLAsource,binar,origN,low60,var_threshold,minmax,LR_60
5,UCLAsource,binar,origN,low80,var_threshold,minmax,LR_80
6,UCLAsource,binar,origN,low100,var_threshold,minmax,LR_100
7,UCLAsource,binar,origN,low120,var_threshold,minmax,LR_120
8,UCLAsource,binar,origN,low140,var_threshold,minmax,LR_140
9,UCLAsource,binar,origN,low160,var_threshold,minmax,LR_160


In [ ]:
num_epoch = 100

for epoch in range(2, num_epoch):

    grid_cv = StratifiedKFold(n_splits=10,
                              shuffle=True,
                              random_state=0)

    eval_cv = StratifiedKFold(n_splits=10,
                              shuffle=True,
                              random_state=epoch)

    
    
    pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
    #pipe.plan_table
    
    pipe.get_results('Data/dti/', caching_steps=['Data', 'Weighters', 'Normalizers', 'Featurizers'],
                 scoring=['roc_auc'], results_file = "Results/Ver3.6/result_" + str(epoch) + ".csv")

No previous results found.
Line: 1/19
Line: 2/19
Line: 3/19
Line: 4/19
